In [1]:
import os
from zipfile import ZipFile
import ms3 # pip install git+https://github.com/johentsch/ms3@corpus_structure
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
features_path = os.path.abspath('features')

## Loading a note list from a zip file

In [2]:
for entry in os.scandir(features_path):
    if not entry.is_file():
        continue
    # load_tsv is a wrapper around pd.read_csv() that converts the columns to their correct types
    notes = ms3.load_tsv(ZipFile(entry.path).open('notes.tsv'))

    # ornamental notes come with duration 0;
    # this function grants them half of their nominal duration
    notes = ms3.add_weighted_grace_durations(notes, weight=0.5)
    
    # making a pd.IntervalIndex
    notes_ix = ms3.replace_index_by_intervals(notes)
    break
notes_ix

,quarterbeats,duration_qb,mc,mn,mc_onset,mn_onset,timesig,staff,voice,duration,gracenote,nominal_duration,scalar,tied,tpc,midi,name,octave,chord_id
interval,,,,,,,,,,,,,,,,,,,
"[0.0, 2.0)",0,2.0,1,1,0,0,4/4,8,1,1/2,NaN,1/2,1,<NA>,3,45,A2,2,14
"[0.0, 2.0)",0,2.0,1,1,0,0,4/4,7,1,1/2,NaN,1/2,1,<NA>,4,52,E3,3,12
"[0.0, 2.0)",0,2.0,1,1,0,0,4/4,6,1,1/2,NaN,1/2,1,<NA>,3,57,A3,3,10
"[0.0, 2.0)",0,2.0,1,1,0,0,4/4,5,1,1/2,NaN,1/2,1,<NA>,0,60,C4,4,8
"[0.0, 2.0)",0,2.0,1,1,0,0,4/4,4,1,1/2,NaN,1/2,1,<NA>,-1,65,F4,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[20.0, 20.5)",20,0.5,6,6,0,0,4/4,5,1,1/8,NaN,1/8,1,-1,0,60,C4,4,236
"[20.0, 20.5)",20,0.5,6,6,0,0,4/4,4,1,1/8,NaN,1/8,1,-1,-1,65,F4,4,235
"[20.0, 20.5)",20,0.5,6,6,0,0,4/4,3,1,1/8,NaN,1/8,1,-1,0,72,C5,5,234


Example of how `midi` abstracts from the information contained in `tpc` + `octave`

| pitch | tpc | octave | midi | pitch                           |
|-------|-----|--------|------|---------------------------------|
| Db4   | -5  | 4      | 61   | C#4, Db4, B##3, Ebbb4...        |
| C#4   | 7   | 4      | 61   | C#4, Db4, B##3, Ebbb4...        |
| C4    | 0   | 4      | 60   | C4, B#3, A###3, Dbb4, Ebbbb4... |
| B#3   | 12  | 3      | 60   | C4, B#3, A###3, Dbb4, Ebbbb4... |
| B3    | 5   | 3      | 59   | B3, Cb4, A##3, Dbbb4...         |

In [3]:
SELECTED_FEATURES = [ # this is just a useful pre-selection but may be further reduced
    'tpc',          # tonal pitch class (note name) where -1 = F; 0 = C; 1 = G; 7 = C#; 5 = Db
    'octave',       # pitch class + octave -> pitch
    'midi',         # ("reduced") pitch as number of the piano key where 60 = C4; 61 = C#4/Db4...
    'quarterbeats', # note's offset from the piece's beginning, measured in quarter notesr
    'duration_qb',  # note's duration measured in quarter notes
    'mn',           # measure/bar
    'mn_onset',     # note's offset from the measure's beginning given as fractions.Fraction
    'timesig',      # time signature as string (because 6/8 is not the same as 3/4)
    'staff',        # instrument layer, starting with the topmost as 1
    'gracenote',]   # if the note is ornamental, the type of ornament

## Slicing
### The current method (slow)

The function `slice_df` relies on the interval index, first computing the intervals for all slices and then checking for each slice, which notes overlap with it.

This approach is extemely slow because of all the comparisons.

In [4]:
CHUNK_SIZE = 1/36 # ideally divisible by 2 and 3

interval2chunk = ms3.slice_df(notes_ix[SELECTED_FEATURES], quarters_per_slice=CHUNK_SIZE)
for interval, chunk in interval2chunk.items():
    print(interval)
    display(chunk)
    break

[0.0, 0.027777777777777776)


,tpc,octave,midi,quarterbeats,duration_qb,mn,mn_onset,timesig,staff,gracenote
"[0.0, 0.027777777777777776)",3,2,45,0.0,0.027778,1,0,4/4,8,NaN
"[0.0, 0.027777777777777776)",4,3,52,0.0,0.027778,1,0,4/4,7,NaN
"[0.0, 0.027777777777777776)",3,3,57,0.0,0.027778,1,0,4/4,6,NaN
"[0.0, 0.027777777777777776)",0,4,60,0.0,0.027778,1,0,4/4,5,NaN
"[0.0, 0.027777777777777776)",-1,4,65,0.0,0.027778,1,0,4/4,4,NaN
"[0.0, 0.027777777777777776)",0,5,72,0.0,0.027778,1,0,4/4,3,NaN
"[0.0, 0.027777777777777776)",-1,5,77,0.0,0.027778,1,0,4/4,2,NaN
"[0.0, 0.027777777777777776)",0,6,84,0.0,0.027778,1,0,4/4,1,NaN


### The faster method

Instead, we could use Map-Reduce to split each note (MAP) into its time slices and combine the results for all time slices (REDUCE).

The following is just a few sketches in this direction (to be discussed) and would need to be adapted.

Ideally we wouldn't be using an interval index (compute with numpy only?) so let's use the notes without:

In [5]:
notes

,quarterbeats,duration_qb,mc,mn,mc_onset,mn_onset,timesig,staff,voice,duration,gracenote,nominal_duration,scalar,tied,tpc,midi,name,octave,chord_id
0,0,2.0,1,1,0,0,4/4,8,1,1/2,NaN,1/2,1,<NA>,3,45,A2,2,14
1,0,2.0,1,1,0,0,4/4,7,1,1/2,NaN,1/2,1,<NA>,4,52,E3,3,12
2,0,2.0,1,1,0,0,4/4,6,1,1/2,NaN,1/2,1,<NA>,3,57,A3,3,10
3,0,2.0,1,1,0,0,4/4,5,1,1/2,NaN,1/2,1,<NA>,0,60,C4,4,8
4,0,2.0,1,1,0,0,4/4,4,1,1/2,NaN,1/2,1,<NA>,-1,65,F4,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,20,0.5,6,6,0,0,4/4,5,1,1/8,NaN,1/8,1,-1,0,60,C4,4,236
236,20,0.5,6,6,0,0,4/4,4,1,1/8,NaN,1/8,1,-1,-1,65,F4,4,235
237,20,0.5,6,6,0,0,4/4,3,1,1/8,NaN,1/8,1,-1,0,72,C5,5,234
238,20,0.5,6,6,0,0,4/4,2,1,1/8,NaN,1/8,1,-1,-1,77,F5,5,233


Express note offsets and endings (i.e. the index of the slice after the last sounding slice) in terms of the index T of all slices:

In [14]:
note_endings = notes.quarterbeats + notes.duration_qb
note_endings_T = (note_endings // CHUNK_SIZE).astype(int)
note_offsets_T = (notes.quarterbeats // CHUNK_SIZE).astype(int)
pd.DataFrame({'start_ix': note_offsets_T,
              'end_ix': note_endings_T
             })

,start_ix,end_ix
0,0,72
1,0,72
2,0,72
3,0,72
4,0,72
...,...,...
235,720,738
236,720,738
237,720,738
238,720,738


Find the last slice and create an index for all slices.

In [7]:
n_slices = note_endings_T.max()
slice_index = pd.Index(range(n_slices))
slice_index

RangeIndex(start=0, stop=738, step=1)

Now we can go ahead and create a binary matrix where the column index corresponds to the note table's index and has the value True for all slices over which a note is active.

In [8]:
def note2activated_slices(start, end):
    return pd.Series(1, 
                     index=range(start, end), 
                     dtype=bool).reindex(slice_index, 
                                         fill_value=False) 

T_ranges_zip = zip(note_offsets_T.values, note_endings_T.values)
note_index_matrix = pd.concat([note2activated_slices(start, end) for start, end in T_ranges_zip], 
                               axis=1)
note_index_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239
0,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

Transposed, it would be closer to thinking of a plot where the time slices are on the x-axis:

In [9]:
note_index_matrix.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

From here we can get easily to all kinds of representations based on the note list, but as final representation it might make sense to stick to the logic of a feature matrix with one slice per row.

For example, we can get the midi pitches active in each slice:

In [10]:
notes.midi

0      45
1      52
2      57
3      60
4      65
       ..
235    60
236    65
237    72
238    77
239    84
Name: midi, Length: 240, dtype: Int64

In [11]:
active_midi_per_slice = pd.concat([notes.midi] * n_slices, 
                                  axis=1, 
                                  ignore_index=True).where(note_index_matrix.T)
active_midi_per_slice

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737
0,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [12]:
def piano_roll(active_midi_per_slice):
    slice_midi_activation = (note_slice.unique() for _, note_slice in active_midi_per_slice.items())
    
    def make_piano_activation(slice_columns):
        return pd.Series(range(128)).isin(next(slice_midi_activation))

    return active_midi_per_slice.apply(make_piano_activation).astype(int)
    
piano_roll(active_midi_per_slice).iloc[:, :80]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


This "piano roll" matrix has 128 piano keys (midi) on the y-axis and on the x-axis we see for every slice which keys are sounding.